In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 16
tree_depth = 10
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.220763206481934 | KNN Loss: 6.225732326507568 | BCE Loss: 1.9950306415557861
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.19043254852295 | KNN Loss: 6.225588321685791 | BCE Loss: 1.9648442268371582
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.231579780578613 | KNN Loss: 6.225534439086914 | BCE Loss: 2.006045341491699
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.17219352722168 | KNN Loss: 6.225142955780029 | BCE Loss: 1.947050929069519
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.164810180664062 | KNN Loss: 6.2255425453186035 | BCE Loss: 1.939267635345459
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.082189559936523 | KNN Loss: 6.224427700042725 | BCE Loss: 1.857762098312378
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.12572956085205 | KNN Loss: 6.224025726318359 | BCE Loss: 1.9017040729522705
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.135915756225586 | KNN Loss: 6.2237677574157715 | BCE Loss: 1.9121475219

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 5.893028736114502 | KNN Loss: 4.714121341705322 | BCE Loss: 1.1789073944091797
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.67885160446167 | KNN Loss: 4.536438465118408 | BCE Loss: 1.1424131393432617
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.617745876312256 | KNN Loss: 4.462981224060059 | BCE Loss: 1.1547646522521973
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 5.47523307800293 | KNN Loss: 4.33170223236084 | BCE Loss: 1.1435308456420898
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.381700038909912 | KNN Loss: 4.258389949798584 | BCE Loss: 1.1233100891113281
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 5.270651817321777 | KNN Loss: 4.141578674316406 | BCE Loss: 1.129073143005371
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.185419082641602 | KNN Loss: 4.051194667816162 | BCE Loss: 1.13422429561615
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.1508612632751465 | KNN Loss: 4.007626533508301 | BCE Loss: 1.1

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.242801666259766 | KNN Loss: 3.2004213333129883 | BCE Loss: 1.0423803329467773
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.2983222007751465 | KNN Loss: 3.217681884765625 | BCE Loss: 1.0806403160095215
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.241363048553467 | KNN Loss: 3.191906452178955 | BCE Loss: 1.0494565963745117
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.24920129776001 | KNN Loss: 3.2198221683502197 | BCE Loss: 1.0293792486190796
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.245300769805908 | KNN Loss: 3.216764211654663 | BCE Loss: 1.0285365581512451
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.269352912902832 | KNN Loss: 3.2253353595733643 | BCE Loss: 1.0440176725387573
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.278356552124023 | KNN Loss: 3.2230496406555176 | BCE Loss: 1.0553069114685059
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 4.317556381225586 | KNN Loss: 3.2745354175567627 | BC

Epoch 32 / 500 | iteration 10 / 30 | Total Loss: 4.19987678527832 | KNN Loss: 3.17140793800354 | BCE Loss: 1.0284690856933594
Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.165718078613281 | KNN Loss: 3.1408562660217285 | BCE Loss: 1.0248615741729736
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.24215030670166 | KNN Loss: 3.179236650466919 | BCE Loss: 1.062913417816162
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.17344856262207 | KNN Loss: 3.1509287357330322 | BCE Loss: 1.022519588470459
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.247107982635498 | KNN Loss: 3.177555561065674 | BCE Loss: 1.0695523023605347
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.196765899658203 | KNN Loss: 3.199018716812134 | BCE Loss: 0.9977471232414246
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.234078407287598 | KNN Loss: 3.1877129077911377 | BCE Loss: 1.0463652610778809
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.192921161651611 | KNN Loss: 3.1578729152679443 | BCE Loss:

Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 4.209124565124512 | KNN Loss: 3.176114320755005 | BCE Loss: 1.0330100059509277
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.183102607727051 | KNN Loss: 3.1572511196136475 | BCE Loss: 1.0258512496948242
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.148543834686279 | KNN Loss: 3.1326045989990234 | BCE Loss: 1.0159392356872559
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.208381175994873 | KNN Loss: 3.163536787033081 | BCE Loss: 1.044844388961792
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.18538761138916 | KNN Loss: 3.1472864151000977 | BCE Loss: 1.0381011962890625
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.159329414367676 | KNN Loss: 3.1228044033050537 | BCE Loss: 1.0365251302719116
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.176938056945801 | KNN Loss: 3.125596761703491 | BCE Loss: 1.0513412952423096
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.148064136505127 | KNN Loss: 3.1323349475860596 | BCE L

...

Epoch 448 / 500 | iteration 15 / 30 | Total Loss: 4.087451457977295 | KNN Loss: 3.0844593048095703 | BCE Loss: 1.0029921531677246
Epoch 448 / 500 | iteration 20 / 30 | Total Loss: 4.091189861297607 | KNN Loss: 3.087977170944214 | BCE Loss: 1.003212571144104
Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 4.084078311920166 | KNN Loss: 3.0850090980529785 | BCE Loss: 0.9990692138671875
Epoch   449: reducing learning rate of group 0 to 1.6100e-07.
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.108673095703125 | KNN Loss: 3.070420265197754 | BCE Loss: 1.038252592086792
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.087719440460205 | KNN Loss: 3.068009853363037 | BCE Loss: 1.019709587097168
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 4.104234218597412 | KNN Loss: 3.080981492996216 | BCE Loss: 1.0232527256011963
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 4.097179889678955 | KNN Loss: 3.0841450691223145 | BCE Loss: 1.0130349397659302
Epoch 449 / 500 | iteration 20 / 30 |

Epoch 459 / 500 | iteration 5 / 30 | Total Loss: 4.105681419372559 | KNN Loss: 3.0701873302459717 | BCE Loss: 1.035494327545166
Epoch 459 / 500 | iteration 10 / 30 | Total Loss: 4.093100547790527 | KNN Loss: 3.058084726333618 | BCE Loss: 1.0350158214569092
Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 4.093588829040527 | KNN Loss: 3.0786702632904053 | BCE Loss: 1.014918565750122
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.063531875610352 | KNN Loss: 3.0682661533355713 | BCE Loss: 0.9952654838562012
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.095851421356201 | KNN Loss: 3.084588050842285 | BCE Loss: 1.011263370513916
Epoch   460: reducing learning rate of group 0 to 1.1270e-07.
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 4.03083610534668 | KNN Loss: 3.045865774154663 | BCE Loss: 0.9849704504013062
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 4.088153839111328 | KNN Loss: 3.0413007736206055 | BCE Loss: 1.0468533039093018
Epoch 460 / 500 | iteration 10 / 30 | 

Epoch 469 / 500 | iteration 25 / 30 | Total Loss: 4.071141242980957 | KNN Loss: 3.0663435459136963 | BCE Loss: 1.0047974586486816
Epoch 470 / 500 | iteration 0 / 30 | Total Loss: 4.078019618988037 | KNN Loss: 3.0741524696350098 | BCE Loss: 1.0038670301437378
Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 4.061554908752441 | KNN Loss: 3.0299925804138184 | BCE Loss: 1.0315624475479126
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.113236427307129 | KNN Loss: 3.0986287593841553 | BCE Loss: 1.0146076679229736
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.0313591957092285 | KNN Loss: 3.0465540885925293 | BCE Loss: 0.9848049879074097
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 4.147325038909912 | KNN Loss: 3.1089487075805664 | BCE Loss: 1.0383764505386353
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 4.082969665527344 | KNN Loss: 3.075295925140381 | BCE Loss: 1.0076736211776733
Epoch   471: reducing learning rate of group 0 to 7.8888e-08.
Epoch 471 / 500 | iteration 0 

Epoch 480 / 500 | iteration 10 / 30 | Total Loss: 4.089734077453613 | KNN Loss: 3.046983242034912 | BCE Loss: 1.042750597000122
Epoch 480 / 500 | iteration 15 / 30 | Total Loss: 4.052148818969727 | KNN Loss: 3.0352730751037598 | BCE Loss: 1.016875982284546
Epoch 480 / 500 | iteration 20 / 30 | Total Loss: 4.065838813781738 | KNN Loss: 3.047142267227173 | BCE Loss: 1.0186963081359863
Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 4.11040735244751 | KNN Loss: 3.1161696910858154 | BCE Loss: 0.9942377805709839
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.094037055969238 | KNN Loss: 3.091932535171509 | BCE Loss: 1.0021045207977295
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.109869956970215 | KNN Loss: 3.0841615200042725 | BCE Loss: 1.0257084369659424
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 4.079349517822266 | KNN Loss: 3.076646327972412 | BCE Loss: 1.0027031898498535
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 4.082265853881836 | KNN Loss: 3.038120746612549 |

Epoch 491 / 500 | iteration 0 / 30 | Total Loss: 4.123971939086914 | KNN Loss: 3.0755860805511475 | BCE Loss: 1.0483858585357666
Epoch 491 / 500 | iteration 5 / 30 | Total Loss: 4.075064182281494 | KNN Loss: 3.0567729473114014 | BCE Loss: 1.0182912349700928
Epoch 491 / 500 | iteration 10 / 30 | Total Loss: 4.0984954833984375 | KNN Loss: 3.062119483947754 | BCE Loss: 1.0363757610321045
Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 4.119644641876221 | KNN Loss: 3.0852420330047607 | BCE Loss: 1.03440260887146
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.0983662605285645 | KNN Loss: 3.0622236728668213 | BCE Loss: 1.0361427068710327
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.060647964477539 | KNN Loss: 3.026803731918335 | BCE Loss: 1.033844232559204
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 4.0682172775268555 | KNN Loss: 3.0446863174438477 | BCE Loss: 1.0235308408737183
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 4.107405662536621 | KNN Loss: 3.10675168037414

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.6970,  3.1057,  2.7477,  3.4826,  3.7074,  0.7246,  2.7431,  2.0297,
          2.1912,  1.7263,  2.2342,  1.8351,  0.9432,  1.6760,  1.1199,  1.4433,
          2.3246,  3.3025,  2.7007,  2.4342,  1.5816,  3.0688,  2.2299,  2.5344,
          1.9169,  1.8646,  2.2183,  1.4689,  1.4033,  0.3506, -0.2870,  1.0944,
          0.3384,  0.9997,  1.2990,  1.5376,  1.1457,  3.5365,  0.7669,  1.3839,
          1.0160, -0.7720, -0.3782,  2.4335,  1.7112,  0.5022, -0.3516,  0.0256,
          1.3029,  2.5729,  1.9033,  0.2487,  1.4146,  0.5848, -0.5768,  1.2612,
          1.6574,  1.1411,  1.4519,  1.8085,  0.7603,  0.7590,  0.2704,  1.9292,
          1.4704,  1.3470, -1.8991,  0.3703,  2.5268,  2.0931,  2.6446,  0.3866,
          1.0189,  2.6251,  2.0826,  1.2561,  0.2715,  0.7285,  0.2521,  1.5650,
          0.0886,  0.5018,  1.9238, -0.3570,  0.3787, -1.0158, -2.3823, -0.4354,
          0.5506, -1.8535,  0.3873, -0.0160, -0.5293, -1.0027,  0.3250,  1.4222,
         -0.8213, -0.8191,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 85.35it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 019 | Total loss: 9.621 | Reg loss: 0.012 | Tree loss: 9.621 | Accuracy: 0.000000 | 0.821 sec/iter
Epoch: 00 | Batch: 001 / 019 | Total loss: 9.618 | Reg loss: 0.011 | Tree loss: 9.618 | Accuracy: 0.000000 | 0.784 sec/iter
Epoch: 00 | Batch: 002 / 019 | Total loss: 9.617 | Reg loss: 0.010 | Tree loss: 9.617 | Accuracy: 0.000000 | 0.775 sec/iter
Epoch: 00 | Batch: 003 / 019 | Total loss: 9.615 | Reg loss: 0.009 | Tree loss: 9.615 | Accuracy: 0.000000 | 0.772 sec/iter
Epoch: 00 | Batch: 004 / 019 | Total loss: 9.613 | Reg loss: 0.009 | Tree loss: 9.613 | Accuracy: 0.000000 | 0.769 sec/iter
Epoch: 00 | Batch: 005 / 019 | Total loss: 9.612 | Reg loss: 0.008 | Tree loss: 9.612 | Accuracy: 0.000000 | 0.768 sec/iter
Epoch: 00 | Batch: 006 / 019 | Total loss: 9.609 | Reg loss: 0.007 | Tree loss: 9.609 | Accuracy: 0.000000 | 0.768 

Epoch: 03 | Batch: 001 / 019 | Total loss: 9.569 | Reg loss: 0.005 | Tree loss: 9.569 | Accuracy: 0.048828 | 0.785 sec/iter
Epoch: 03 | Batch: 002 / 019 | Total loss: 9.567 | Reg loss: 0.005 | Tree loss: 9.567 | Accuracy: 0.056641 | 0.784 sec/iter
Epoch: 03 | Batch: 003 / 019 | Total loss: 9.568 | Reg loss: 0.005 | Tree loss: 9.568 | Accuracy: 0.066406 | 0.784 sec/iter
Epoch: 03 | Batch: 004 / 019 | Total loss: 9.568 | Reg loss: 0.005 | Tree loss: 9.568 | Accuracy: 0.048828 | 0.784 sec/iter
Epoch: 03 | Batch: 005 / 019 | Total loss: 9.567 | Reg loss: 0.005 | Tree loss: 9.567 | Accuracy: 0.048828 | 0.783 sec/iter
Epoch: 03 | Batch: 006 / 019 | Total loss: 9.563 | Reg loss: 0.005 | Tree loss: 9.563 | Accuracy: 0.054688 | 0.783 sec/iter
Epoch: 03 | Batch: 007 / 019 | Total loss: 9.562 | Reg loss: 0.006 | Tree loss: 9.562 | Accuracy: 0.044922 | 0.783 sec/iter
Epoch: 03 | Batch: 008 / 019 | Total loss: 9.560 | Reg loss: 0.006 | Tree loss: 9.560 | Accuracy: 0.058594 | 0.783 sec/iter
Epoch: 0

Epoch: 06 | Batch: 004 / 019 | Total loss: 9.509 | Reg loss: 0.009 | Tree loss: 9.509 | Accuracy: 0.083984 | 0.78 sec/iter
Epoch: 06 | Batch: 005 / 019 | Total loss: 9.500 | Reg loss: 0.009 | Tree loss: 9.500 | Accuracy: 0.091797 | 0.78 sec/iter
Epoch: 06 | Batch: 006 / 019 | Total loss: 9.498 | Reg loss: 0.010 | Tree loss: 9.498 | Accuracy: 0.097656 | 0.78 sec/iter
Epoch: 06 | Batch: 007 / 019 | Total loss: 9.502 | Reg loss: 0.010 | Tree loss: 9.502 | Accuracy: 0.070312 | 0.78 sec/iter
Epoch: 06 | Batch: 008 / 019 | Total loss: 9.494 | Reg loss: 0.010 | Tree loss: 9.494 | Accuracy: 0.074219 | 0.78 sec/iter
Epoch: 06 | Batch: 009 / 019 | Total loss: 9.492 | Reg loss: 0.011 | Tree loss: 9.492 | Accuracy: 0.091797 | 0.78 sec/iter
Epoch: 06 | Batch: 010 / 019 | Total loss: 9.480 | Reg loss: 0.011 | Tree loss: 9.480 | Accuracy: 0.070312 | 0.78 sec/iter
Epoch: 06 | Batch: 011 / 019 | Total loss: 9.480 | Reg loss: 0.011 | Tree loss: 9.480 | Accuracy: 0.076172 | 0.78 sec/iter
Epoch: 06 | Batc

Epoch: 09 | Batch: 007 / 019 | Total loss: 9.191 | Reg loss: 0.015 | Tree loss: 9.191 | Accuracy: 0.076172 | 0.785 sec/iter
Epoch: 09 | Batch: 008 / 019 | Total loss: 9.187 | Reg loss: 0.016 | Tree loss: 9.187 | Accuracy: 0.064453 | 0.785 sec/iter
Epoch: 09 | Batch: 009 / 019 | Total loss: 9.160 | Reg loss: 0.016 | Tree loss: 9.160 | Accuracy: 0.058594 | 0.786 sec/iter
Epoch: 09 | Batch: 010 / 019 | Total loss: 9.128 | Reg loss: 0.016 | Tree loss: 9.128 | Accuracy: 0.050781 | 0.786 sec/iter
Epoch: 09 | Batch: 011 / 019 | Total loss: 9.126 | Reg loss: 0.017 | Tree loss: 9.126 | Accuracy: 0.054688 | 0.786 sec/iter
Epoch: 09 | Batch: 012 / 019 | Total loss: 9.113 | Reg loss: 0.017 | Tree loss: 9.113 | Accuracy: 0.052734 | 0.786 sec/iter
Epoch: 09 | Batch: 013 / 019 | Total loss: 9.087 | Reg loss: 0.017 | Tree loss: 9.087 | Accuracy: 0.068359 | 0.787 sec/iter
Epoch: 09 | Batch: 014 / 019 | Total loss: 9.061 | Reg loss: 0.018 | Tree loss: 9.061 | Accuracy: 0.050781 | 0.787 sec/iter
Epoch: 0

Epoch: 12 | Batch: 010 / 019 | Total loss: 8.543 | Reg loss: 0.020 | Tree loss: 8.543 | Accuracy: 0.068359 | 0.804 sec/iter
Epoch: 12 | Batch: 011 / 019 | Total loss: 8.519 | Reg loss: 0.020 | Tree loss: 8.519 | Accuracy: 0.056641 | 0.804 sec/iter
Epoch: 12 | Batch: 012 / 019 | Total loss: 8.532 | Reg loss: 0.020 | Tree loss: 8.532 | Accuracy: 0.041016 | 0.804 sec/iter
Epoch: 12 | Batch: 013 / 019 | Total loss: 8.509 | Reg loss: 0.021 | Tree loss: 8.509 | Accuracy: 0.056641 | 0.804 sec/iter
Epoch: 12 | Batch: 014 / 019 | Total loss: 8.482 | Reg loss: 0.021 | Tree loss: 8.482 | Accuracy: 0.058594 | 0.805 sec/iter
Epoch: 12 | Batch: 015 / 019 | Total loss: 8.450 | Reg loss: 0.021 | Tree loss: 8.450 | Accuracy: 0.048828 | 0.805 sec/iter
Epoch: 12 | Batch: 016 / 019 | Total loss: 8.454 | Reg loss: 0.022 | Tree loss: 8.454 | Accuracy: 0.058594 | 0.805 sec/iter
Epoch: 12 | Batch: 017 / 019 | Total loss: 8.426 | Reg loss: 0.022 | Tree loss: 8.426 | Accuracy: 0.035156 | 0.804 sec/iter
Epoch: 1

...

Epoch: 84 | Batch: 015 / 019 | Total loss: 3.508 | Reg loss: 0.023 | Tree loss: 3.508 | Accuracy: 0.087891 | 0.861 sec/iter
Epoch: 84 | Batch: 016 / 019 | Total loss: 3.541 | Reg loss: 0.023 | Tree loss: 3.541 | Accuracy: 0.078125 | 0.861 sec/iter
Epoch: 84 | Batch: 017 / 019 | Total loss: 3.577 | Reg loss: 0.023 | Tree loss: 3.577 | Accuracy: 0.082031 | 0.861 sec/iter
Epoch: 84 | Batch: 018 / 019 | Total loss: 3.550 | Reg loss: 0.023 | Tree loss: 3.550 | Accuracy: 0.062718 | 0.861 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 85 | Batch: 000 / 019 | Total loss: 3.553 | Reg loss: 0.023 | Tree loss: 3.553 | Accuracy: 0.111328 | 0.862 sec/iter
Epoch: 85 | Batch: 001 / 019 | Total loss: 3.538 | Reg loss: 0.023 | Tree loss: 3.538 | Ac

Epoch: 87 | Batch: 018 / 019 | Total loss: 3.557 | Reg loss: 0.023 | Tree loss: 3.557 | Accuracy: 0.090592 | 0.861 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 88 | Batch: 000 / 019 | Total loss: 3.543 | Reg loss: 0.023 | Tree loss: 3.543 | Accuracy: 0.083984 | 0.862 sec/iter
Epoch: 88 | Batch: 001 / 019 | Total loss: 3.555 | Reg loss: 0.023 | Tree loss: 3.555 | Accuracy: 0.048828 | 0.862 sec/iter
Epoch: 88 | Batch: 002 / 019 | Total loss: 3.495 | Reg loss: 0.023 | Tree loss: 3.495 | Accuracy: 0.107422 | 0.862 sec/iter
Epoch: 88 | Batch: 003 / 019 | Total loss: 3.537 | Reg loss: 0.023 | Tree loss: 3.537 | Accuracy: 0.097656 | 0.862 sec/iter
Epoch: 88 | Batch: 004 / 019 | Total loss: 3.551 | Reg loss: 0.023 | Tree loss: 3.551 | Ac

Epoch: 91 | Batch: 000 / 019 | Total loss: 3.546 | Reg loss: 0.023 | Tree loss: 3.546 | Accuracy: 0.066406 | 0.862 sec/iter
Epoch: 91 | Batch: 001 / 019 | Total loss: 3.534 | Reg loss: 0.023 | Tree loss: 3.534 | Accuracy: 0.068359 | 0.862 sec/iter
Epoch: 91 | Batch: 002 / 019 | Total loss: 3.515 | Reg loss: 0.023 | Tree loss: 3.515 | Accuracy: 0.080078 | 0.862 sec/iter
Epoch: 91 | Batch: 003 / 019 | Total loss: 3.517 | Reg loss: 0.023 | Tree loss: 3.517 | Accuracy: 0.080078 | 0.862 sec/iter
Epoch: 91 | Batch: 004 / 019 | Total loss: 3.542 | Reg loss: 0.023 | Tree loss: 3.542 | Accuracy: 0.078125 | 0.862 sec/iter
Epoch: 91 | Batch: 005 / 019 | Total loss: 3.552 | Reg loss: 0.023 | Tree loss: 3.552 | Accuracy: 0.074219 | 0.862 sec/iter
Epoch: 91 | Batch: 006 / 019 | Total loss: 3.511 | Reg loss: 0.023 | Tree loss: 3.511 | Accuracy: 0.095703 | 0.862 sec/iter
Epoch: 91 | Batch: 007 / 019 | Total loss: 3.538 | Reg loss: 0.023 | Tree loss: 3.538 | Accuracy: 0.080078 | 0.862 sec/iter
Epoch: 9

Epoch: 94 | Batch: 003 / 019 | Total loss: 3.493 | Reg loss: 0.023 | Tree loss: 3.493 | Accuracy: 0.085938 | 0.862 sec/iter
Epoch: 94 | Batch: 004 / 019 | Total loss: 3.516 | Reg loss: 0.023 | Tree loss: 3.516 | Accuracy: 0.062500 | 0.862 sec/iter
Epoch: 94 | Batch: 005 / 019 | Total loss: 3.518 | Reg loss: 0.023 | Tree loss: 3.518 | Accuracy: 0.072266 | 0.862 sec/iter
Epoch: 94 | Batch: 006 / 019 | Total loss: 3.524 | Reg loss: 0.023 | Tree loss: 3.524 | Accuracy: 0.083984 | 0.862 sec/iter
Epoch: 94 | Batch: 007 / 019 | Total loss: 3.530 | Reg loss: 0.023 | Tree loss: 3.530 | Accuracy: 0.074219 | 0.862 sec/iter
Epoch: 94 | Batch: 008 / 019 | Total loss: 3.536 | Reg loss: 0.023 | Tree loss: 3.536 | Accuracy: 0.064453 | 0.862 sec/iter
Epoch: 94 | Batch: 009 / 019 | Total loss: 3.517 | Reg loss: 0.023 | Tree loss: 3.517 | Accuracy: 0.062500 | 0.862 sec/iter
Epoch: 94 | Batch: 010 / 019 | Total loss: 3.532 | Reg loss: 0.023 | Tree loss: 3.532 | Accuracy: 0.085938 | 0.862 sec/iter
Epoch: 9

Epoch: 97 | Batch: 006 / 019 | Total loss: 3.495 | Reg loss: 0.023 | Tree loss: 3.495 | Accuracy: 0.087891 | 0.863 sec/iter
Epoch: 97 | Batch: 007 / 019 | Total loss: 3.530 | Reg loss: 0.023 | Tree loss: 3.530 | Accuracy: 0.074219 | 0.863 sec/iter
Epoch: 97 | Batch: 008 / 019 | Total loss: 3.489 | Reg loss: 0.023 | Tree loss: 3.489 | Accuracy: 0.074219 | 0.863 sec/iter
Epoch: 97 | Batch: 009 / 019 | Total loss: 3.492 | Reg loss: 0.023 | Tree loss: 3.492 | Accuracy: 0.078125 | 0.863 sec/iter
Epoch: 97 | Batch: 010 / 019 | Total loss: 3.519 | Reg loss: 0.023 | Tree loss: 3.519 | Accuracy: 0.078125 | 0.863 sec/iter
Epoch: 97 | Batch: 011 / 019 | Total loss: 3.505 | Reg loss: 0.023 | Tree loss: 3.505 | Accuracy: 0.087891 | 0.863 sec/iter
Epoch: 97 | Batch: 012 / 019 | Total loss: 3.522 | Reg loss: 0.023 | Tree loss: 3.522 | Accuracy: 0.068359 | 0.863 sec/iter
Epoch: 97 | Batch: 013 / 019 | Total loss: 3.510 | Reg loss: 0.023 | Tree loss: 3.510 | Accuracy: 0.087891 | 0.863 sec/iter
Epoch: 9

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 10.0


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 1024


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

============== Pattern 232 ==============
============== Pattern 233 ==============
============== Pattern 234 ==============
============== Pattern 235 ==============
============== Pattern 236 ==============
============== Pattern 237 ==============
============== Pattern 238 ==============
============== Pattern 239 ==============
============== Pattern 240 ==============
============== Pattern 241 ==============
============== Pattern 242 ==============
============== Pattern 243 ==============
============== Pattern 244 ==============
============== Pattern 245 ==============
============== Pattern 246 ==============
============== Pattern 247 ==============
============== Pattern 248 ==============
============== Pattern 249 ==============
============== Pattern 250 ==============
============== Pattern 251 ==============
============== Pattern 252 ==============
============== Pattern 253 ==============
============== Pattern 254 ==============
============== Pattern 255 =======

============== Pattern 588 ==============
============== Pattern 589 ==============
============== Pattern 590 ==============
============== Pattern 591 ==============
============== Pattern 592 ==============
============== Pattern 593 ==============
============== Pattern 594 ==============
============== Pattern 595 ==============
============== Pattern 596 ==============
============== Pattern 597 ==============
============== Pattern 598 ==============
============== Pattern 599 ==============
============== Pattern 600 ==============
============== Pattern 601 ==============
============== Pattern 602 ==============
============== Pattern 603 ==============
============== Pattern 604 ==============
============== Pattern 605 ==============
============== Pattern 606 ==============
============== Pattern 607 ==============
============== Pattern 608 ==============
============== Pattern 609 ==============
============== Pattern 610 ==============
============== Pattern 611 =======

============== Pattern 891 ==============
============== Pattern 892 ==============
============== Pattern 893 ==============
============== Pattern 894 ==============
============== Pattern 895 ==============
============== Pattern 896 ==============
============== Pattern 897 ==============
============== Pattern 898 ==============
============== Pattern 899 ==============
============== Pattern 900 ==============
============== Pattern 901 ==============
============== Pattern 902 ==============
============== Pattern 903 ==============
============== Pattern 904 ==============
============== Pattern 905 ==============
============== Pattern 906 ==============
============== Pattern 907 ==============
============== Pattern 908 ==============
============== Pattern 909 ==============
============== Pattern 910 ==============
============== Pattern 911 ==============
============== Pattern 912 ==============
============== Pattern 913 ==============
============== Pattern 914 =======